This notebook covers how to use the firebolt ingestion tool via the ResourceManager. 

In [1]:
# support async in notebooks, read more about why we need this here: https://pypi.org/project/nest-asyncio/
import nest_asyncio

nest_asyncio.apply()

### Logging

In [2]:
# configure logging
import logging

logging.basicConfig(
    format="{asctime} - {name} - {levelname} - {message}", style="{", level="INFO"
)

### Connection

The ingestion library requires a connection object from the Firebolt Python SDK. The following cells demonstrate how to create one from the SDK's ResourceManager. Please refer to the SDK documentaiton for additional ways to generate a connection object. If you already have a connection object, you can skip this section!

create a .env file in the same directory as this notebook with the following contents (fill in values):
```
FIREBOLT_USER=''
FIREBOLT_PASSWORD=''
FIREBOLT_SERVER=''
FIREBOLT_DEFAULT_REGION=''
```


In [6]:
from firebolt.service.manager import ResourceManager
from firebolt.common import Settings

rm = ResourceManager()

In [7]:
import time

# optionally replace "" with the database and engine to which you want to connect.
# otherwise, run this as is to create a temp database and engine for demo purposes.
default_name = f"temp_{int(time.time())}"
database_name = "" or default_name
engine_name = "" or default_name
table_name = "" or default_name

database = rm.databases.create(name=database_name, region="us-east-1")
engine = rm.engines.create(name=engine_name)
engine.attach_to_database(database=rm.databases.get_by_name(name=database_name))
engine = engine.start()
connection = engine.get_connection()

2022-03-29 15:05:58,698 - firebolt.service.database - INFO - Creating Database (name=temp_1648591558)
2022-03-29 15:05:59,165 - firebolt.service.engine - INFO - Creating Engine (name=temp_1648591558)
2022-03-29 15:06:00,328 - firebolt.service.binding - INFO - Attaching Engine (engine_id=926b3673-5a5c-42a5-869c-cccbdc80be04, name=temp_1648591558) to Database (database_id=52988ca2-a73d-4631-932c-fe45993d8828, name=temp_1648591558)
2022-03-29 15:06:01,637 - firebolt.model.engine - INFO - Starting Engine (engine_id=926b3673-5a5c-42a5-869c-cccbdc80be04, name=temp_1648591558)
2022-03-29 15:07:47,693 - firebolt.model.engine - INFO - Engine status_summary=ENGINE_STATUS_SUMMARY_STARTING_INITIALIZING
2022-03-29 15:12:24,605 - firebolt.model.engine - INFO - Engine status_summary=ENGINE_STATUS_SUMMARY_RUNNING


### Table Configuration

You can define a table configuration via yaml, or in Python. The following cell shows how to do it first in Python, and then as yaml.

In [12]:
# Temp: delete me
table_name = "temp_12345"

In [27]:
from firebolt_ingest.model.table import Table, Column, Partition, FileType

table = Table(
    table_name=table_name,
    columns=[Column(name="col_1", type="STRING"), Column(name="col_2", type="INT")],
    primary_index=["col_1"],
    file_type=FileType.PARQUET,
    object_pattern=["*.parquet"],
)

In [20]:
import yaml

print(yaml.dump(table.dict()))

columns:
- extract_partition: null
  name: col_1
  type: STRING
- extract_partition: null
  name: col_2
  type: INT
compression: null
file_type: !!python/object/apply:firebolt_ingest.model.table.FileType
- PARQUET
object_pattern:
- '*.parquet'
partitions: []
primary_index: []
table_name: temp_12345



In [23]:
table = Table.parse_yaml(
    """
columns:
- extract_partition: null
  name: col_1
  type: STRING
- extract_partition: null
  name: col_2
  type: INT
compression: null
file_type:
- PARQUET
object_pattern:
- '*.parquet'
partitions: []
primary_index: []
table_name: temp_12345
"""
)

ValidationError: 1 validation error for Table
file_type
  value is not a valid enumeration member; permitted: 'ORC', 'PARQUET', 'TSV' (type=type_error.enum; enum_values=[<FileType.ORC: 'ORC'>, <FileType.PARQUET: 'PARQUET'>, <FileType.TSV: 'TSV'>])

In [28]:
from firebolt_ingest.service.table import TableService

ts = TableService(connection)
ts.create_internal_table(table=table)

OperationalError: Error executing query:
Invalid operation error: SQL query not valid.
SQL query:
==========
CREATE FACT TABLE IF NOT EXISTS temp_12345
(col_1 STRING, col_2 INT, source_file_name STRING, source_file_timestamp DATETIME)
PRIMARY INDEX (col_1)
              [31m^=== ERROR HERE![0m
==========
Error line: 2
Error column: 14
Error message: syntax error, unexpected '('
